In [28]:
import pandas as pd
import numpy as np
import os
import boto3
import botocore
import paramiko

In [29]:
allSRS_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/allSRS.pickle.gz'
sra_dump_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/sra_dump.fastqc.bowtie_algn.pickle'
allSRX_pickle_dir='/cellar/users/btsui/Data/SRA/DUMP/allSRX.pickle.gz'
#
merged_kallisto_run_info_dir='/nrnb/users/btsui/Data/all_seq/rnaseq_merged/merged_kallisto_run_info.pickle'

In [30]:
allSRS_with_processed_data_dir=allSRS_pickle_dir.replace('.pickle.gz','.with_processed_data.pickle.gz')

### export only part of the SRS

In [31]:
allSRS=pd.read_pickle(allSRS_pickle_dir)

In [32]:
sra_dump_df=pd.read_pickle(sra_dump_pickle_dir)

#### find all the processed SRR IDs 

In [33]:
merged_kallisto_run_info_df=pd.read_pickle(merged_kallisto_run_info_dir)

In [34]:
snpProcessedRuns=sra_dump_df.index[sra_dump_df['fastqc']['Total basepairs processed:\s+([0-9,]+)'].notnull()]

#### retain only the SRS with associated with processed SRR IDs

In [35]:
processedSrr=np.union1d(merged_kallisto_run_info_df.Run,snpProcessedRuns)

In [36]:
processedSrs=sra_dump_df[sra_dump_df.index.isin(processedSrr)]['SRAmeta']['Sample'].unique()
srs_a=allSRS.index.get_level_values(0)
srs_m=srs_a.isin(processedSrs)
allSRS_processed=allSRS[srs_m]

In [37]:
allSRS_processed.to_pickle(allSRS_with_processed_data_dir)

In [38]:
#%time pd.read_pickle('/cellar/users/btsui/Data/SRA/DUMP/allSRS.with_processed_data.pickle.gz') 

### upload to SRS

In [39]:
### change the ip of the instance
instance_ip='ec2-34-215-138-8.us-west-2.compute.amazonaws.com'
remoteDir='~/efs/all_seq/meta_data/'

In [40]:
snpBedDirs=['/cellar/users/btsui/Data/dbsnp/snp_beds/Homo_sapiens.bed']

In [41]:
#mergedMatricesDir='/nrnb/users/btsui/Data/all_seq/rnaseq_merged/'
uploadDirs=[sra_dump_pickle_dir,allSRS_pickle_dir,allSRX_pickle_dir,merged_kallisto_run_info_dir,
            allSRS_with_processed_data_dir]+snpBedDirs
inFnameS=pd.Series(uploadDirs)

In [42]:
#include_m1=(inFnameS.str.contains('gene_symbol')&(~inFnameS.str.contains('.gz')))
#include_m2=inFnameS.isin(['merged_kallisto_run_info.pickle'])
transferFnames=inFnameS#[include_m1|include_m2]

In [43]:
key = paramiko.RSAKey.from_private_key_file('/cellar/users/btsui/.ssh/jupyter_hub.pem')
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

In [44]:
#mountEfs='sudo mount -t efs fs-e1636448:/ efs'

#try:
# Here 'ubuntu' is user name and 'instance_ip' is public IP of EC2
client.connect(hostname=instance_ip, username="ec2-user", pkey=key)
# Execute a command(cmd) after connecting/ssh to an instance
cmd='mkdir ~/efs'
stdin, stdout, stderr = client.exec_command(cmd)
cmd='sudo mount -t nfs4 -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 fs-e1636448.efs.us-west-2.amazonaws.com:/ ~/efs'
stdin, stdout, stderr = client.exec_command(cmd)

In [45]:
#client.connect(hostname=instance_ip, username="ec2-user", pkey=key)
remoteDir

'~/efs/all_seq/meta_data/'

In [46]:
cmd='mkdir -p '+remoteDir
stdin, stdout, stderr = client.exec_command(cmd)
print (stdout.read())
print (stderr.read())

b''
b''


In [47]:
myRsyncDirFormat=' rsync  -Pvu -e "ssh -i $HOME/.ssh/jupyter_hub.pem" {localFDir} ec2-user@{instance_ip}:{remoteDir}'

In [48]:
addHostVerificationCmd="ssh -o StrictHostKeyChecking=no -i $HOME/.ssh/jupyter_hub.pem ec2-user@{instance_ip}  ls".format(instance_ip=instance_ip)
os.system(addHostVerificationCmd)


0

In [49]:
#transferFnames.values

In [50]:
for localFname in transferFnames.values:
    localFDir= (localFname)
    rsyncCmd=myRsyncDirFormat.format(localFDir=localFDir,instance_ip=instance_ip,remoteDir=remoteDir+'.')
    print (rsyncCmd)
    print (os.system(rsyncCmd))

 rsync  -Pvu -e "ssh -i $HOME/.ssh/jupyter_hub.pem" /cellar/users/btsui/Data/SRA/DUMP/sra_dump.fastqc.bowtie_algn.pickle ec2-user@ec2-34-215-138-8.us-west-2.compute.amazonaws.com:~/efs/all_seq/meta_data/.
0
 rsync  -Pvu -e "ssh -i $HOME/.ssh/jupyter_hub.pem" /cellar/users/btsui/Data/SRA/DUMP/allSRS.pickle.gz ec2-user@ec2-34-215-138-8.us-west-2.compute.amazonaws.com:~/efs/all_seq/meta_data/.
0
 rsync  -Pvu -e "ssh -i $HOME/.ssh/jupyter_hub.pem" /cellar/users/btsui/Data/SRA/DUMP/allSRX.pickle.gz ec2-user@ec2-34-215-138-8.us-west-2.compute.amazonaws.com:~/efs/all_seq/meta_data/.
0
 rsync  -Pvu -e "ssh -i $HOME/.ssh/jupyter_hub.pem" /nrnb/users/btsui/Data/all_seq/rnaseq_merged/merged_kallisto_run_info.pickle ec2-user@ec2-34-215-138-8.us-west-2.compute.amazonaws.com:~/efs/all_seq/meta_data/.
0
 rsync  -Pvu -e "ssh -i $HOME/.ssh/jupyter_hub.pem" /cellar/users/btsui/Data/SRA/DUMP/allSRS.with_processed_data.pickle.gz ec2-user@ec2-34-215-138-8.us-west-2.compute.amazonaws.com:~/efs/all_seq/meta_

In [23]:
#!rsync  -Pvu -e "ssh -i $HOME/.ssh/jupyter_hub.pem" /cellar/users/btsui/Data/SRA/DUMP/allSRS.with_processed_data.pickle.gz ec2-user@ec2-34-221-100-235.us-west-2.compute.amazonaws.com:~/efs/all_seq/meta_data/.

In [24]:
#!rsync  -Pvu -e "ssh -i $HOME/.ssh/jupyter_hub.pem" /nrnb/users/btsui/Data/all_seq/rnaseq_merged/Canis_familiaris.gene_symbol.tpm.index.txt \
#ec2-user@ec2-52-24-55-248.us-west-2.compute.amazonaws.com:~/.

In [25]:
#!ls $HOME/.ssh/

In [26]:
#import seaborn

In [27]:
#!c